# GitHub on Hacker News trends analysis

In [2]:
from __future__ import print_function
import pandas as pd

## Input parameters

In [9]:
project_id = "your-project-id"
process_date = "2018-12-01"
process_date_nodash = "20181201"

## Exploratory Data Analysis

## Github activity data
- Link: [Data](https://bigquery.cloud.google.com/table/githubarchive:day.20181230) - [More info](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)

### Different event type in Gihub activity
- [Event Types & Payloads](https://developer.github.com/v3/activity/events/types/) explaination

In [6]:
query = """
SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}` 
GROUP BY 1
ORDER BY 2 DESC
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201` 
GROUP BY 1
ORDER BY 2 DESC



,type,cnt
0,PushEvent,588724
1,CreateEvent,155010
2,WatchEvent,67607
3,PullRequestEvent,56635
4,IssueCommentEvent,46972
5,IssuesEvent,27592
6,ForkEvent,24331
7,DeleteEvent,22590
8,PullRequestReviewCommentEvent,9756
9,MemberEvent,5201


### Top 10 repos with the most comments in their issues
- __IssueCommentEvent__: Triggered when an issue comment is created, edited, or deleted.

In [18]:
query = """
SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,cnt
0,google-test/signcla-probe-repo,327
1,Azure/azure-rest-api-specs,287
2,kubernetes/kubernetes,227
3,rust-lang/rust,207
4,apache/spark,204
5,freeCodeCamp/freeCodeCamp,196
6,everypolitician/everypolitician-data,192
7,TeamNewPipe/NewPipe,158
8,openshift/origin,140
9,NixOS/nixpkgs,126


### Top 10 repos by stars and fork event

In [8]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,BcRikko/NES.css,386,35,421
1,leisurelicht/wtfpython-cn,241,31,272
2,satwikkansal/wtfpython,190,30,220
3,cssanimation/css-animation-101,178,5,183
4,firecracker-microvm/firecracker,150,13,163
5,crazyandcoder/kindle_free_books,132,31,163
6,withspectrum/spectrum,132,9,141
7,afshinea/stanford-cs-230-deep-learning,120,17,137
8,algorithm-visualizer/algorithm-visualizer,119,15,134
9,olifolkerd/tabulator,114,3,117


## Hacker News data
- Link: [Data](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.full) - [More info](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)

### Top domains shared in Hacker News
- Domain with higher score are more likely to make it to the front page.
- __nytimes__ has the highest average score.

In [12]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10



,domain,avg_score,cnt
0,github.com,14.966667,30
1,medium.com,15.592593,27
2,www.youtube.com,12.666667,24
3,www.nytimes.com,41.263158,19
4,venturebeat.com,2.100000,10
5,www.reddit.com,21.428571,7
6,www.theguardian.com,31.166667,6
7,en.wikipedia.org,15.833333,6
8,arstechnica.com,22.666667,6
9,www.theverge.com,2.200000,5


### What domains have the best chance of getting more than 40 upvotes?
- Certainly Hacker News likes content hosted on sites like github.com and the nytimes.

In [11]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10



,domain,score_gt_40,cnt
0,www.nytimes.com,4,19
1,github.com,4,30
2,medium.com,3,27
3,www.wsj.com,2,4
4,www.theatlantic.com,2,5
5,www.youtube.com,2,24
6,www.jamiefuller.com,1,1
7,arstechnica.com,1,6
8,www.vulture.com,1,2
9,www.newsshooter.com,1,1


### Top 10 Hacker news stories from Github by highest score

In [17]:
query = """
SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{0}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2018-12-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC



,submitter,story_id,url,score
0,ithinco,18574181,https://github.com/ithinco/i-am-chinese-the-dr...,129
1,mountainview,18576170,https://github.com/YugaByte/yugabyte-db,115
2,oxplot,18575094,https://github.com/oxplot/pdftilecut,64
3,codeadict,18574683,https://github.com/alertlogic/erllambda,64
4,pjmlp,18575802,https://github.com/chocolatey/boxstarter,9
5,snek,18577658,https://github.com/devsnek/engine262,8
6,delvincasper,18577036,https://github.com/jerverless/jerverless,4
7,andrewchaa,18574107,https://github.com/andrewchaa/functional.pipe,4
8,anmonteiro90,18578964,https://github.com/anmonteiro/aws-lambda-ocaml...,4
9,KumarAbhirup,18577887,https://github.com/KumarAbhirup/bulk-mail-cli,4


## Example Final table: GitHub on Hacker News Trends of 2018-12-01

In [20]:
query = """
WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{1}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10
""".format(process_date_nodash, process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2018-12-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10



,date,github_url,github_repo,hn_score,hn_story_id,stars,forks
0,2018-12-01,https://github.com/ithinco/i-am-chinese-the-dr...,ithinco/i-am-chinese-the-dragonfly-must-go-on,129,18574181,60.0,1.0
1,2018-12-01,https://github.com/YugaByte/yugabyte-db,YugaByte/yugabyte-db,115,18576170,2.0,NaN
2,2018-12-01,https://github.com/alertlogic/erllambda,alertlogic/erllambda,64,18574683,48.0,NaN
3,2018-12-01,https://github.com/oxplot/pdftilecut,oxplot/pdftilecut,64,18575094,91.0,NaN
4,2018-12-01,https://github.com/chocolatey/boxstarter,chocolatey/boxstarter,9,18575802,1.0,NaN
5,2018-12-01,https://github.com/devsnek/engine262,devsnek/engine262,8,18577658,1.0,NaN
6,2018-12-01,https://github.com/andrewchaa/functional.pipe,andrewchaa/functional.pipe,4,18574107,2.0,NaN
7,2018-12-01,https://github.com/anmonteiro/aws-lambda-ocaml...,anmonteiro/aws-lambda-ocaml-runtime,4,18578964,5.0,NaN
8,2018-12-01,https://github.com/KumarAbhirup/bulk-mail-cli,None,4,18577887,NaN,NaN
9,2018-12-01,https://github.com/jerverless/jerverless,None,4,18577036,NaN,NaN


## Python PyPI stats
- The Python Software Foundation provides the raw logs of Python installation activitie
- Link: [Data](https://bigquery.cloud.google.com/table/the-psf:pypi.downloads20181230) - [More info](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)

__Challenge__: 
- Find associated Github stars, fork event, and Hacker News story for top downloads Python packages from pip

### Top 10 downloads packages from pip

In [15]:
query = """
SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads{0}`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads20181201`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc



,day,project,downloads
0,2018-12-01,pip,1562226
1,2018-12-01,urllib3,1271997
2,2018-12-01,botocore,1069194
3,2018-12-01,six,966172
4,2018-12-01,python-dateutil,946327
5,2018-12-01,s3transfer,877832
6,2018-12-01,docutils,813135
7,2018-12-01,pyyaml,796706
8,2018-12-01,pyasn1,782540
9,2018-12-01,jmespath,772065


### What is the number stars and fork event for botocore?

In [19]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,boto/botocore,2,1,3


## Resources
- [GitHub data, ready for you to explore with BigQuery](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)
- [Hacker News on BigQuery](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)
- [Analyzing PyPI package downloads](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)